In [ ]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader,CSVLoader

text_loader_kwargs = {'autodetect_encoding': True}
loader = DirectoryLoader(
    "relation_data", 
    glob="**/[!.]*", 
    loader_cls=CSVLoader, 
    silent_errors=True,
    loader_kwargs=text_loader_kwargs, 
    show_progress=True,
    use_multithreading=True, 
    max_concurrency=4,
    exclude=[], 
    recursive=True
)
docs = loader.load()
docs,len(docs)




In [8]:
loader = CSVLoader(file_path="relation_data\get_relation_data.csv",encoding='utf-8')
docs = loader.load()
docs

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:1: SyntaxWarning: invalid escape sequence '\g'
C:\Users\XY-33\AppData\Local\Temp\ipykernel_11540\1221248398.py:1: SyntaxWarning: invalid escape sequence '\g'
  loader = CSVLoader(file_path="relation_data\get_relation_data.csv",encoding='utf-8')


[Document(metadata={'source': 'relation_data\\get_relation_data.csv', 'row': 0}, page_content='relationship: Teslaconomics - PUBLISHED_TWEET -> https://twitter.com/Teslaconomics/status/1695286752758620339'),
 Document(metadata={'source': 'relation_data\\get_relation_data.csv', 'row': 1}, page_content='relationship: August31,2023 - PUBLISH_DATE -> elon-musk-shows-fsd-beta-v12-live-test-drive-on-x'),
 Document(metadata={'source': 'relation_data\\get_relation_data.csv', 'row': 2}, page_content='relationship: GPT-3 - HAS_PARAMETERS -> 1750亿'),
 Document(metadata={'source': 'relation_data\\get_relation_data.csv', 'row': 3}, page_content='relationship: GPT-4 - HAS_PARAMETERS -> 1.8万亿'),
 Document(metadata={'source': 'relation_data\\get_relation_data.csv', 'row': 4}, page_content='relationship: GPT-4 - HAS_CAPABILITY -> 看懂照片'),
 Document(metadata={'source': 'relation_data\\get_relation_data.csv', 'row': 5}, page_content='relationship: GPT-4 - HAS_CAPABILITY -> 编程'),
 Document(metadata={'sourc

In [170]:
from langchain_community.vectorstores import FAISS
from element.qinfanapi import create_llm,create_embeddings
embedding = create_embeddings()

db = FAISS.load_local("./faiss_index", embedding, allow_dangerous_deserialization=True)
query = "ni"
# 执行相似度搜索并合并文档内容
embedding_vector = embedding.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector, k=6)

docs,docs[0]

([Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 153}, page_content='140'),
  Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 246}, page_content='234'),
  Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 157}, page_content='144'),
  Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 13}, page_content='8'),
  Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 117}, page_content='104'),
  Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 7}, page_content='2')],
 Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 153}, page_content='140'))

In [176]:
def combine_documents(documents):
    return "\n".join(doc.page_content for doc in documents)

docs = db.similarity_search_with_score(query, k=3)
# combined_content_docs = combine_documents(docs)
print(docs)
# combined_content_docs

[(Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 153}, page_content='140'), 0.29903734), (Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 246}, page_content='234'), 0.3012036), (Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 157}, page_content='144'), 0.30602056)]


In [169]:
retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5})  # 构建检索器
docs = retriever.get_relevant_documents(query)
docs

C:\Users\XY-33\AppData\Local\Temp\ipykernel_10888\3713021052.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


[Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 153}, page_content='140'),
 Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 246}, page_content='234'),
 Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 157}, page_content='144'),
 Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 13}, page_content='8')]

In [1]:

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
def combine_documents(documents):
    return "\n".join(doc.page_content for doc in documents)

loader = PyPDFLoader("data/Inner Mastery, Outer Impact_Hitendra Wadhwa_liber3.pdf", extract_images=True)
pages = loader.load_and_split()
texts = combine_documents(pages)
text_splitter = RecursiveCharacterTextSplitter(texts,chunk_size=200, chunk_overlap=20)
documents = text_splitter.split_documents(documents=pages)
len(documents)

IndexError: list index out of range

In [97]:
documents

[Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 0}, page_content='拐点\n1.119.510 下载采取样子\n万维钢 著\nTURNINGPOINT\n时间\n台海出版社站在AI颠覆世界的前夜人\n类\n进\n程'),
 Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 1}, page_content='拐点\n站在AI颠覆世界的前夜\n万维钢著\n台海出版社'),
 Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 2}, page_content='有时候几十年过去了什么都没发生；有时候几个星期就发生了几十年的事。(There\naredecadeswherenothinghappens;andthereareweekswheredecades\nhappen.)\n————列宁\n关于这东西为什么好使我们不提供任何解释，问就是上天的眷顾。(Weofferno'),
 Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 2}, page_content='explanationastowhyanythingworksexceptdivinebenevolence.)\n————山姆·阿尔特曼'),
 Document(metadata={'source': 'data\\万维钢：拐点—站在AI颠覆世界的前夜 copy.pdf', 'page': 3}, page_content='CONTENTS\n目录\n前言:人要比AI凶/1\nChatGPT究竟是什么\n03底牌和命门:AI能力的局限/034\n04数学:Al视角下的语义和智能/048\n当AI进入人类社会\n03战略:AI商业的竞争趋势/076\n04社会:被AI“接管”后的忧患/089\n01大变局：一个新智慧形态的产生/002\n02“开悟”和“涌现”:AI能力的三个境界/014\n01效率:

In [156]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader,CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader

def combine_documents(documents):
    return "\n".join(doc.page_content for doc in documents)

# text_loader_kwargs = {'autodetect_encoding': True}
text_loader_kwargs = {'encoding': 'utf-8'}
loader = DirectoryLoader("data", glob="*.csv", loader_cls=CSVLoader, silent_errors=True,
        loader_kwargs=text_loader_kwargs, show_progress=True,
        use_multithreading=False, max_concurrency=4,
        exclude=[], recursive=False)
docs = loader.load()
texts = combine_documents(docs)
text_splitter = RecursiveCharacterTextSplitter(texts,chunk_size=500, chunk_overlap=20)
documents = text_splitter.split_documents(documents=docs)
len(documents),documents

100%|██████████| 1/1 [00:00<00:00,  8.70it/s]


(8664,
 [Document(metadata={'source': 'data\\get_relation_data1.csv', 'row': 0}, page_content='relationship: 老龄化 - ASSOCIATED_WITH -> 经济减速'),
  Document(metadata={'source': 'data\\get_relation_data1.csv', 'row': 1}, page_content='relationship: 老龄化 - ASSOCIATED_WITH -> 家庭负担增加'),
  Document(metadata={'source': 'data\\get_relation_data1.csv', 'row': 2}, page_content='relationship: 老龄化 - ASSOCIATED_WITH -> 个人发展机会减少'),
  Document(metadata={'source': 'data\\get_relation_data1.csv', 'row': 3}, page_content='relationship: 养老观念 - BELONGS_TO -> 中国人'),
  Document(metadata={'source': 'data\\get_relation_data1.csv', 'row': 4}, page_content='relationship: 养老方式 - BELONGS_TO -> 中国人'),
  Document(metadata={'source': 'data\\get_relation_data1.csv', 'row': 5}, page_content='relationship: 今天的中国老年人 - CHANGE_TO -> 不仅仅是负担，而是能发挥很多积极能量的群体'),
  Document(metadata={'source': 'data\\get_relation_data1.csv', 'row': 6}, page_content='relationship: 老年人 - IS_NOT -> 一穷二白，守着两亩薄田过活的人'),
  Document(metadata={'source': 'da

In [4]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def combine_documents(documents):
    return "\n".join(doc.page_content for doc in documents)

loader = PyMuPDFLoader("data/Inner Mastery, Outer Impact_Hitendra Wadhwa_liber3.pdf")  # 最快的 PDF 解析选项，但不能提取图片中的文字
pages = loader.load_and_split()
texts = combine_documents(pages)
text_splitter = RecursiveCharacterTextSplitter(texts,chunk_size=200, chunk_overlap=20)
documents = text_splitter.split_documents(documents=pages)
len(documents)

4222

In [3]:
documents

[Document(metadata={'source': 'data/Inner Mastery, Outer Impact_Hitendra Wadhwa_liber3.pdf', 'file_path': 'data/Inner Mastery, Outer Impact_Hitendra Wadhwa_liber3.pdf', 'page': 3, 'total_pages': 234, 'format': 'PDF 1.5', 'title': 'Aspose', 'author': 'Aspose', 'subject': 'Aspose', 'keywords': '', 'creator': 'Aspose Pty Ltd.', 'producer': 'Aspose.PDF for Java 24.7', 'creationDate': 'D:20241009054813Z', 'modDate': "D:20241009054827+00'00'", 'trapped': ''}, page_content='Copyright © 2022 by Hitendra Wadhwa\nJacket design by Terri Sirma\nJacket illustrati'),
 Document(metadata={'source': 'data/Inner Mastery, Outer Impact_Hitendra Wadhwa_liber3.pdf', 'file_path': 'data/Inner Mastery, Outer Impact_Hitendra Wadhwa_liber3.pdf', 'page': 3, 'total_pages': 234, 'format': 'PDF 1.5', 'title': 'Aspose', 'author': 'Aspose', 'subject': 'Aspose', 'keywords': '', 'creator': 'Aspose Pty Ltd.', 'producer': 'Aspose.PDF for Java 24.7', 'creationDate': 'D:20241009054813Z', 'modDate': "D:20241009054827+00'00'"

In [44]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader(file_path="text.txt", encoding='utf-8')
docs = loader.load()
docs

[Document(metadata={'source': 'text.txt'}, page_content='一提到老龄化，很多人会联想到经济减速，家庭负担增加，个人发展机会减少。我想告诉你，这其实是一个思维误区，这和我们中国人传统的养老观念、养老方式有关。\n如果和过去中国几千年的传统社会做纵向比较，你就会发现，今天的中国老年人已经有了三个显著的变化，这些变化都指向一个方向，就是老年人不仅仅是负担，而是能发挥很多积极能量的群体。这三个变化是什么呢？\n第一，老年人不再只是一穷二白，守着两亩薄田过活的人。\n和过去中国历史任何一个时期相比，我们这个时代的老人肯定更有钱，这是不言而喻的。这个钱可能是房子，也可能是退休金，甚至有可能是家里面农村的宅基地。即使没有这些，也有国家兜底的养老金。无论多少，肯定总体上要比以前更好。\n这是大面上的，还需要注意，这群老年人还有很多是富人。按照今年的统计，中国60岁以上的高净值人群，占这个群体的5%，已经很高了。什么是高净值人群呢？就是那些可投资资产在1000万以上的富人。也就是说，很多老人已经财富自由了。\n当今社会不是有一种被诟病的现象吗？就是大批的青年人“躺平”或者“啃老”，“啃老”的前提条件就是有一大批可啃的老人，这些老人就是先富起来的老人。\n')]